In [1]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

def save_classification_report(predictions, labels, output_dir):
    report = classification_report(
        labels, predictions,
        target_names=["conclusion", "premise"]
    )
    
    with open(f"{output_dir}/classification_report.txt", "w") as f:
        f.write(report)
    
    return report

def plot_confusion_matrix(y_true, y_pred, output_path):
    cm = confusion_matrix(y_true, y_pred)
    
    labels = np.array([
        ['(TP)', '(FN)'],
        ['(FP)', '(TN)']
    ])
    
    # Create annotation array properly
    annot = np.empty_like(labels, dtype=object)
    for i in range(labels.shape[0]):
        for j in range(labels.shape[1]):
            annot[i, j] = f"{labels[i, j]}\n{cm[i, j]}"
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=annot, fmt='', cmap="Blues",
                xticklabels=["Predicted Conclusion", "Predicted Premise"],
                yticklabels=["Actual Conclusion", "Actual Premise"])
    plt.xlabel('Prediction')
    plt.ylabel('Ground Truth')
    plt.title('Confusion Matrix for Argumentative Classification')
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

# Main processing code
folder_path = 'predictions'
all_data = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        all_data.append(df[['actual_class', 'predicted_label']])

# Concatenate all data
all_data_df = pd.concat(all_data, ignore_index=True)

# Map actual classes for consistency
all_data_df['actual_class_mapped'] = all_data_df['actual_class'].map({
    'prem': 'premise', 
    'conc': 'conclusion'
})

# Generate reports
output_dir = '.'
report = save_classification_report(
    all_data_df['predicted_label'], 
    all_data_df['actual_class_mapped'], 
    output_dir
)

plot_confusion_matrix(
    all_data_df['actual_class_mapped'],
    all_data_df['predicted_label'],
    "confusion_matrix.png"
)
